In [46]:
import os
import requests
import pandas as pd

api_key = os.environ.get('riot_api_key')
base_url = "https://na1.api.riotgames.com"
solo_duo = "RANKED_SOLO_5x5"

print(api_key)
print(base_url)

RGAPI-f39807c0-1dc0-4741-a2b4-9953668d5da7
https://na1.api.riotgames.com


In [47]:
def getChallengers() -> pd.DataFrame:
	url = f"{base_url}/lol/league/v4/challengerleagues/by-queue/{solo_duo}?api_key={api_key}"

	response = requests.get(url)

	if response.status_code == 200:
			data = response.json()
			players = data['entries']

			return pd.DataFrame(players)


In [48]:
def getMatches(puuid) :
	url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=4&start=0&count=20&api_key={api_key}"

	response = requests.get(url)

	print(response)

	if response.status_code == 200:
			data = response.json()
			print(data)

In [49]:
challengers = getChallengers()

puuid = challengers.iloc[0]["puuid"]

getMatches(puuid=puuid)

<Response [200]>
[]
